<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Backpropagation Practice

## *Data Science Unit 4 Sprint 2 Assignment 2*

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
##### Your Code Here #####
import numpy as np

inputs = np.array([[0,0,1,0,1,1,0],
                 [0,1,0,1,0,1,0],
                 [1,1,1,0,0,1,0]])

correct_output = [[0],[1],[1],[1],[1],[0],[0]]

In [2]:
inputs.T.shape

(7, 3)

In [3]:
class NeuralNetwork:
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

        # Initial Weights
        # 3x4 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputs, self.hiddenNodes)
       
        # 4x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1)
        
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2)
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        
        return self.activated_output
    
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o 
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(self.output_sum) 
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T) # dot product of matriz size 10000x1 and 1x1500 yeilds 10000x1500
        
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.hidden_sum) # yeilds 10000x1500 
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta) # yeilds 784x1500
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) # yeilds  1500x1
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)

In [4]:
nn = NeuralNetwork()

In [5]:
for i in range(10000):
    nn.train(inputs.T, correct_output)

In [6]:
nn.activated_output

array([[0.00760863],
       [0.9892332 ],
       [0.98879693],
       [0.99999797],
       [0.99983822],
       [0.00932644],
       [0.00223566]])

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


### Parts
1. Gathering & Transforming the Data
2. Making MNIST a Binary Problem
3. Estimating your Neural Network (the part you focus on)

### Gathering the Data 

`keras` has a handy method to pull the mnist dataset for you. You'll notice that each observation is a 28x28 arrary which represents an image. Although most Neural Network frameworks can handle higher dimensional data, that is more overhead than necessary for us. We need to flatten the image to one long row which will be 784 values (28X28). Basically, you will be appending each row to one another to make on really long row. 

In [3]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [4]:
# input image dimensions
img_rows, img_cols = 28, 28

In [5]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
x_train = x_train[:10000]
x_test = x_test[:1000]
y_train = y_train[:10000]
y_test = y_test[:1000]

In [7]:
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# Normalize Our Data
x_train = x_train / 255
x_test = x_test / 255

In [8]:
# X Variable Types
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

In [9]:
# Now the data should be in a format you're more familiar with
x_train.shape

(10000, 784)

### Making MNIST a Binary Problem 
MNIST is multiclass classification problem; however we haven't covered all the necessary techniques to handle this yet. You would need to one-hot encode the target, use a different loss metric, and use softmax activations for the last layer. This is all stuff we'll cover later this week, but let us simply the problem for now: Zero or all else.

In [10]:
import numpy as np

y_temp = np.zeros(y_train.shape)
y_temp[np.where(y_train == 0.0)[0]] = 1
y_train = y_temp
y_temp = np.zeros(y_test.shape)
y_temp[np.where(y_test == 0.0)[0]] = 1
y_test = y_temp

In [11]:
# A Nice Binary target for ya to work with
y_train = y_train.reshape(y_train.shape[0],1)
y_test = y_test.reshape(y_test.shape[0],1)

In [12]:
y_train.shape

(10000, 1)

In [13]:
y_test.shape

(1000, 1)

In [14]:
# X Variable Types
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

### Estimating Your `net

In [15]:
##### Your Code Here #####
class Neural_Network(object):
    def __init__(self):
        # Set up Architecture of Neural Network
        self.inputSize = 784
        self.hiddenNodes = 785
        self.outputNodes = 1

        # Initial Weights
        # 784x1500 Matrix Array for the First Layer
        self.weights1 = np.random.rand(self.inputSize, self.hiddenNodes)
       
        # 1500x1 Matrix Array for Hidden to Output
        self.weights2 = np.random.rand(self.hiddenNodes, self.outputNodes)
        
    def sigmoid(self, s):
        return 1 / (1+np.exp(-s))
    
    def sigmoidPrime(self, s):
        sx = self.sigmoid(s)
        return sx * (1 - sx)
    
    def feed_forward(self, X):
        """
        Calculate the NN inference using feed forward.
        aka "predict"
        """
        
        # Weighted sum of inputs => hidden layer
        self.hidden_sum = np.dot(X, self.weights1) # dot product of X(input) 10000x784 and weights1 784x1500 yeilds 10000x1500 matrix    
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum) 
        
        # Weight sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.weights2) # 10000x1500 matrix by 1500x1 matrix yeilds 10000x1
        
        # Final activation of output
        self.activated_output = self.sigmoid(self.output_sum) 
        
        return self.activated_output
    
        
    def backward(self, X,y,o):
        """
        Backward propagate through the network
        """
        
        # Error in Output
        self.o_error = y - o 
        
        # Apply Derivative of Sigmoid to error
        # How far off are we in relation to the Sigmoid f(x) of the output
        # ^- aka hidden => output
        self.o_delta = self.o_error * self.sigmoidPrime(self.output_sum) 
        
        # z2 error
        self.z2_error = self.o_delta.dot(self.weights2.T) # dot product of matriz size 10000x1 and 1x1500 yeilds 10000x1500
        
        
        # How much of that "far off" can explained by the input => hidden
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.hidden_sum) # yeilds 10000x1500 
        
        # Adjustment to first set of weights (input => hidden)
        self.weights1 += X.T.dot(self.z2_delta) # yeilds 784x1500
        # Adjustment to second set of weights (hidden => output)
        self.weights2 += self.activated_hidden.T.dot(self.o_delta) # yeilds  1500x1
        

    def train(self, X, y):
        o = self.feed_forward(X)
        self.backward(X,y,o)


In [16]:
nn = Neural_Network()

In [17]:
for i in range(50):
    nn.train(x_train, y_train)

In [18]:
nn.o_error

array([[-1.],
       [ 0.],
       [-1.],
       ...,
       [-1.],
       [-1.],
       [-1.]])

In [59]:
y_train

array([[0.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

## Stretch Goals: 

- Make MNIST a multiclass problem using cross entropy & soft-max
- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?